![](./title.png)
- 本文提出了HIP-NN模型，用量化计算数据建模分子性质。
- 并且可以将分子属性，例如能量，分解成求和项。

<img src="./fig1.png" align="left" width="45%"></img>
<div>
    <li>绿色是相互作用层interaction，从周围原子获取信息，有两个</li>
    <li>红色的是on-site layer，每组有三层</li>
    <li>蓝色是每一层的能量</li>
    <li>原子特征有80个</li>
使用theano框架
</div>




## A 原子表达
分子的构型可以用$\mathcal{C}=\{(Z_i,r_i)\}$表达,$Z_i$是原子序数，$r_i$是原子坐标，但使用原子间距$r_{ij}=|r_i-r_j|$

相邻原子条件$r_{ij}<R_{cut}$

使用独热向量表示$Z_i$

$z_{i,a}^0=\delta_{Z_i},\mathcal{Z}_{a}$

## B 原子特征和能量
原子特征：$z_{i,a}^l$

$l$是层索引；$a$是特征索引；$i$是原子索引

输入特征向量：$z_i^0$

下一层的特征：$z_i^{l+1}$是基于上一层相邻原子的特征得来的$(z_j^l,r_{ij})$

$\hat{E}_i^n=\sum_{a=1}^{N_{feature}}w_a^nz_{i,a}^{l_n}+b^n $

$w_a^n和b^n是学习参数$，总能量：$E\approx \hat{E}=\sum _{i=1}^{N_{atom}}\hat{E}_i$

$\hat{E}_i=\sum _{n=0}^{N_{interaction}}\ \hat{E}_i^n$

## C On-site layers
on-site layer操作每一个原子特征

$\tilde{z}_{i,a}^{l+1}=f( \sum_{b}W_{ab}^lz_{i,b}^l+B_a^l )$

$W,B$是学习参数，$f$是激活函数 $f(x)=log(1+e^x)$

使用残差网络

$z_{i,a}^{l+1}=\sum_{b}(\tilde{W} _{ab}^l\tilde{z}_{i,b}^{l+1}+M_{ab}^lz_{i,b}^l)+\tilde B_a^l$

$W,M,B$都是学习参数

## D 相互作用层
$\tilde z_{i,a}^{l+1}=f(\sum_{j,b}v _{ab}^l(r_{ij})z_{j,b}^l+\sum_{b}W _{ab}^l\tilde{z}_{i,b}^l+B_a^l) \tag{8}$

$v _{ab}^l(r_{ij})$是从相邻原子收集到的信息。

$v _{ab}^l(r_{ij})=\sum_{\nu }V_{\nu,ab}^ls_\nu^l(r_{ij}) \tag{9}$

$V$是学习参数，s是空间灵敏度参数，有20个$\nu$

$s_\nu^l(r_{ij})=exp\left [ -\frac{(r^{-1}-\mu_{\nu,l}^{-1})^2}{2\sigma _{\nu,l}^{-2}}\right]\varphi _{cut}(r) \tag{10}$

$\mu_{\nu,l}$和$\sigma _{\nu,l}$是学习参数，$R_{cut}$从1.7到10bohr

$\varphi _{cut}(r)=\left\{\begin{matrix}
 \left [ \cos (\frac{\pi}{2}\frac{r}{R_{cut}}  ) \right ]^2  & r\le R_{cut} \\
 0 & r\ge  R_{cut}
\end{matrix}\right. \tag{11}$

![](./敏感度.png)
- 蓝色曲线 不同$\mu$的初始灵敏度
- 黑色虚线 所有敏感度由于截断函数限制的范围

## 损失函数
$\mathcal{L}=\frac{1}{\sigma _E}(MAE+RMSE)+\mathcal{L}_{L2}+\mathcal{L}_R \tag{14}$

$\sigma _E=\sqrt[]{\left \langle (E-\left \langle E \right \rangle )^2 \right \rangle }_D \tag{15}$

$\mathcal{L}_{L2}=\lambda _{L2}\left ( \frac{||w||_2^2}{\sigma _E^2} + ||W||_2^2+||V||_2^2+||\tilde W||_2^2+||\tilde M||_2^2\right) \tag{16}$

其中$\lambda _{L2}$是一个很小的超参数，能够减小结果的偏离程度

$\mathcal{L}_R=\lambda _R\left \langle R \right \rangle _D \tag{17}$

为了增加能量项的层次加入的

$R=\sum _{n=1}^{N_{interaction}}\sum _{i=1}^{N_{atom}}\frac{(\hat{E}_i^n)^2}{(\hat{E}_i^n)^2+(\hat{E}_i^{n-1})^2} \tag{18}$

正常情况下能量会迅速衰减，当R比较大的时候，说明模型不正常

## 随机优化
使用`Adam`优化器

用$U=\{w,b,W,B,V,\sigma ,\mu ,\hat{W},\hat{B},\hat{M}\}$表达模型参数

用测试集的损失打分(bset_score)，分数不变时减小学习率，分数减小停止训练

## 讨论
在QM9和MD轨迹都有很好的表现。

有以下原因：

1. 使用灵敏度函数，使用距离导数作为参数。短范围内高灵敏度，长范围低灵敏度。
2. 使用ResNet

少量的L2正则化(Eq.(16))对有助于稳定均方根误差，且对MAE的影响很小。

能量分层的机制提升了HIPNN的表现。

缺点：预测的能量虽然有改进，但是有些方面预测误差还是比较大。

![](./title2.png)
- 将HIPNN和PYSEQM结合起来
- HIPNN根据结构预测半经验参数
- PYSEQM根据半经验参数和结构计算能量

![](./fig-2-2.png)
- PYSEQM是一个开源的半经验量化包，提供了包含PM3在内的几种量化方法。

PYSEQM将分子结构和A中生成的半经验参数作为输入产生核心哈密顿量和库伦积分(橘色块)，然后调用SCF程序获得化学哈密顿量(棕色块)，然后去预测各种化学性质，对比DFT计算结果作为`损失`来训练模型。

$E_{tot}=E_{elec}+\sum_{i<j}E_{nuc,ij}+E_{D3H4}$

总能量=电子能+核间库伦作用+氢键，色散能量矫正

$E_{elec}=Tr[D(h+H)]/2$

D：单粒子密度矩阵，h：核心哈密顿；H：Fock矩阵

$H(D)=h+G(D)$

G：库伦矩阵

PM3的参数（18）
    
|参数|物理含义|
|---|---|
|$U_{ss}$|s轨道的on-site energy|
|$U_{pp}$|p轨道的on-site energy|
|$\xi_s$|s轨道的径向指数 S7|
|$\xi_p$|p轨道的径向指数|
|$\beta_s$|s轨道的共振参数|
|$\beta_p$|p轨道的共振参数|
|$g_{ss}$|以原子为中心s轨道的库伦积分项(ss,ss)|
|$g_{sp}$|以原子为中心s和p轨道的库伦积分项(ss,pp)|
|$g_{pp}$|原子p轨道的库伦积分项($p_ip_i$,$p_jp_j$),(i, j=x,y,z, i ≠j) |
|$g_{p2}$|原子p轨道的库伦积分项($p_ip_i$,$p_ip_i$),(i = j) |
|$h_{sp}$|原子s和p轨道的交换库伦积分项(sp,sp)|
|$\alpha$|核相互作用中的指数项|
|$K_1$,$K_2$|核相互作用中高斯项的权重项|
|$L_1$,$L_2$|核相互作用中高斯项的收缩项|
|$M_1$,$M_2$|核相互作用中高斯项的中心项|